In [3]:
import pandas as pd

In [7]:
df = pd.read_csv("filmler_clean.csv")

In [9]:
df.head()

,film adi,ozet ve detaylar,tur,oyuncular,yonetmen,vizyon tarihi,sure,clean
0,Bergen,Arabeskin unutulmayan kadın seslerinden biri o...,"['Dram', 'Biyografik']","['Farah Zeynep Abdullah', 'Erdal Beşikçioğlu',...","['Mehmet Binay', 'Caner Alper']",4 Mart 2022,2s 26dk,arabesk unutmak kadın ses o Bergen inmek hayat...
1,Kirpi Sonic 2,Çocuk filmi \n\n\nKirpi Sonic filminin devam h...,"['Animasyon', 'Aile', 'Macera']","['Jim Carrey', 'James Marsden', None]",['Jeff Fowler'],8 Nisan 2022,2s 2dk,çocuk film kirpi Sonic film deva halka o yapım...
2,Fantastik Canavarlar: Dumbledore'un Sırları,"Karanlık büyücü Gellert Grindelwald, büyücülük...","['Fantastik', 'Macera']","['Eddie Redmayne', 'Jude Law', 'Mads Mikkelsen']",['David Yates'],15 Nisan 2022,2s 22dk,Karanlık büyü Gellert Grindelwald büyü Dünya k...
3,Aykut Enişte 2,"Aykut Enişte 2, sevdiği kadınla evlilik arifes...",['Komedi'],"['Cem Gelinoğlu', 'Melis Babadağ', 'Hakan Yılm...",['Onur Bilgetay'],3 Aralık 2021,1s 55dk,Aykut enişte 2 sevmek kadın ev arife kendi tür...
4,Morbius,"Morbius, hastalığına deva bulmaya çalışırken b...","['Aksiyon', 'Fantastik', 'Macera']","['Jared Leto', 'Matt Smith (XI)', 'Adria Arjona']",['Daniel Espinosa'],1 Nisan 2022,1s 45dk,Morbius hastalık deva bulmak Çalışır bambaşka ...


In [8]:
clean = df["clean"]

In [10]:
len(clean)

5261

In [11]:
word2idx = {}
idx2word = {}

In [12]:
i = 0
for cell in clean:
    words = cell.split()
    for word in words:
        if word in word2idx:
            continue
        word2idx[word] = i
        idx2word[i] = word
        i += 1

In [13]:
import numpy as np

In [14]:
film_sayisi = len(clean)
vocab_size = len(word2idx)

In [15]:
filmler = np.zeros((film_sayisi, vocab_size))

In [16]:
for i, cell in enumerate(clean):
    words = cell.split()
    for word in words:
        wordIndex = word2idx[word]
        filmler[i][wordIndex] += 1

In [17]:
doc_frequencies = {}

In [18]:
for word in word2idx:
    doc_count = 0
    for film in clean:
        if word in film:
            doc_count += 1
    
    doc_frequencies[word] = doc_count

In [32]:
sorted_frequencies = sorted(doc_frequencies, key=doc_frequencies.get, reverse=True)

In [33]:
for i in range(5):
  sorted_frequencies[i]=0



In [35]:
 
  print(sorted_frequencies[2:10])

[0, 0, 0, 'an', 'er', 'ü', 'bi', 'ya']


In [23]:
for word in sorted_frequencies[-10:]:
    print(word, doc_frequencies[word])

Daddy 1
menü 1
hindi 1
kaçırıldığında 1
labaratuarla 1
Labaratuarda 1
1621 1
Peluchonneau 1
şairane 1
bitecekmiş 1


In [36]:
filmler_tf_idf = np.zeros((film_sayisi, vocab_size))
film_sayisi = len(clean)

for i, film in enumerate(filmler):
    if i % 1000 == 0:
        print(f"{i} inci film işleniyor...")
    for j, tf in enumerate(film):
        idf = 0
        if tf > 0:
            idf = np.log(film_sayisi / (doc_frequencies[idx2word[j]] + 1)) + 1
        tf_idf_score = tf * idf
        filmler_tf_idf[i][j] = tf_idf_score

0 inci film işleniyor...
1000 inci film işleniyor...
2000 inci film işleniyor...
3000 inci film işleniyor...
4000 inci film işleniyor...
5000 inci film işleniyor...


In [37]:
print(filmler_tf_idf[0][:10])
print(filmler[0][:10])

[15.91727698  5.6762561   2.64346401  3.71300448  2.00038019 81.81782041
  7.50567832  7.43327815  9.68137717  1.43536991]
[ 2.  1.  1.  1.  2. 10.  4.  4.  2.  1.]


In [38]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [39]:
def film_2_str(row_index):
    film = df.iloc[row_index]
    adi = film['film adi']
    turu = film['tur']
    ozet = film['ozet ve detaylar']
    if len(ozet) > 150:
        ozet = ozet[:150] + "..."

    return f"Film adı: {adi} \nTürü: {turu} \nÖzeti: {ozet}"

In [40]:
def most_similar_films(film_adi, benzer_film_sayisi):
    film_indexi = df[df["film adi"] == film_adi].index[0]
    print("Aranan Film:")
    print(film_2_str(film_indexi))
    print()
    print("* " * 30)
    print()

    benzerlikler = []
    sorgu_film = filmler_tf_idf[film_indexi]
    for vec in filmler_tf_idf:
        benzerlik = cos_sim(sorgu_film, vec)
      
        benzerlikler.append(benzerlik)
    k=0
    en_benzer_film_indexleri = (- np.array(benzerlikler)).argsort()[:10]
    for h  in range(10): 
      film = df.iloc[en_benzer_film_indexleri[h]] 
      filmasıl = df.iloc[film_indexi] 
      if film['tur']==filmasıl['tur']: 
        if k<5:
          en_benzer_film_indexleri[k]=en_benzer_film_indexleri[h] 
          k=k+1
    
    print("Benzer Filmler:\n")
    for i, film_idx in enumerate(en_benzer_film_indexleri[1:4]):
        print(str(i+1) + ") " + film_2_str(film_idx))
        print()
        print("- " * 20)
        print()

In [41]:
df.iloc[:10]["film adi"]

0                                         Bergen
1                                  Kirpi Sonic 2
2    Fantastik Canavarlar: Dumbledore'un Sırları
3                                 Aykut Enişte 2
4                                        Morbius
5                                        Kırmızı
6                                            Kar
7                    Kesişme: İyi ki Varsın Eren
8                     Örümcek-Adam Eve Dönüş Yok
9                                     The Batman
Name: film adi, dtype: object

In [53]:
most_similar_films("Fantastik Canavarlar: Dumbledore'un Sırları", 3)

Aranan Film:
Film adı: Fantastik Canavarlar: Dumbledore'un Sırları 
Türü: ['Fantastik', 'Macera'] 
Özeti: Karanlık büyücü Gellert Grindelwald, büyücülük dünyasının kontrolünü ele geçirmek için harekete geçmiştir. Bunu öğrenen Profesör Albus Dumbledore, önc...

* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

Benzer Filmler:

1) Film adı: Fantastik Canavarlar: Grindelwald'ın Suçları 
Türü: ['Fantastik', 'Macera'] 
Özeti: Newt Scamander'ın da yardımıyla Amerika Birleşik Devletleri Sihir Kongresi (MACUSA) tarafından yakalanan karanlık büyücü Gellert Grindelwald kaçmayı b...

- - - - - - - - - - - - - - - - - - - - 

2) Film adı: Fantastik Canavarlar Nelerdir, Nerede Bulunurlar? 
Türü: ['Fantastik', 'Macera'] 
Özeti: JK Rowling'in aynı isimli fantastik ansiklopedik kitabına dayanan Fantastik Canavarlar Nelerdir, Nerede Bulunurlar? filmi fantastik yaratıklar hakkınd...

- - - - - - - - - - - - - - - - - - - - 

3) Film adı: Harry Potter ve Ölüm Yadigarları: Bölüm 2 
Türü: ['Fan